# PySpark / Ibis / Delta Lake

In [1]:
import pyspark
from delta import *
import ibis

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-332-delta-230/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e9c3584-51ed-4dc9-9484-30e53318fe4e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 105ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |  

23/05/05 22:37:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Create a Delta table

In [4]:
df = spark.createDataFrame([(0, "Bob", 75), (1, "Sue", 25), (2, "Jim", 27)]).toDF(
    "id", "name", "age"
)

In [5]:
df.write.format("delta").save("tmp/fun_people")

23/05/05 22:37:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Append to the Delta table

In [6]:
df = spark.createDataFrame([(8, "Larry", 19), (9, "Jerry", 69)]).toDF(
    "id", "name", "age"
)

In [7]:
df.write.format("delta").mode("append").save("tmp/fun_people")

## Create an Ibis table from the Delta table

In [9]:
con = ibis.pyspark.connect(spark)

In [11]:
spark.read.format("delta").load("tmp/fun_people").createOrReplaceTempView("fun_people")

In [12]:
table = con.table("fun_people")

In [13]:
table.filter(table.age >= 50).execute()

,id,name,age
0,9,Jarry,69
1,8,Larry,75


In [14]:
table.head(2).execute()

,id,name,age
0,9,Jarry,69
1,1,Sue,25


## Time travel back to version 0 of the data

In [15]:
spark.read.format("delta").load("tmp/fun_people").createOrReplaceTempView("fun_people_v0")

In [16]:
table_v0 = con.table("fun_people_v0")

In [17]:
table_v0.filter(table_v0.age >= 50).execute()

,id,name,age
0,9,Jarry,69
1,8,Larry,75


In [18]:
table_v0.head(2).execute()

,id,name,age
0,9,Jarry,69
1,1,Sue,25
